# ECommerce Data Set Preparation
# by Pranab Sachithanandan (psachi@utexas.edu)

### The goal of this project is to explore the data to understand the current status of the business (descriptive modeling), analyze any trends, and strategize long term customer engagement strategies through insights.
### The two data tables used are Customers (customerID, Acquisition Date, Acquisition Channel, Acquisition Device) and Funnel (customerID, Class Name, SKUs Viewed, SKUs Added to Cart, SKUs Purchased, Revenue Generated).

### First, let's set up the environment, import the cleaned data, get it ready for analysis, join the tables together, and export it to be easily analyzed in Excel.

In [1]:
import pandas as pd # dataframes
import numpy as np # arrays and a few useful math functions
import matplotlib.pyplot as plot # graphs!
import seaborn as sns # visualize dataframes easily and beautifully
import gc # force garbage collection when doing ad-hoc analyses
%matplotlib inline 

df_customers = pd.read_excel('Customers.xlsx')
df_funnel = pd.read_excel('Funnel.xlsx')
df_customers.head() #lookin' good?

,customerID,Acquisition Date,Acquisition Channel,Acquisition Device
0,21,2008-06-14,Search - Paid,Desktop
1,546,2007-08-16,Search - Paid,Desktop
2,1153,2011-02-15,NaN,Desktop
3,1721,2006-12-23,Search - Paid,Desktop
4,2155,2011-05-25,Search - Paid,Desktop


In [2]:
df_customers.info() # characterize the data. note: there were 19 duplicate customerID values initially (1402 total)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1383 entries, 0 to 1382
Data columns (total 4 columns):
customerID             1383 non-null int64
Acquisition Date       1383 non-null datetime64[ns]
Acquisition Channel    960 non-null object
Acquisition Device     618 non-null object
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 43.3+ KB


In [3]:
# reformat the column data types to be more useful
df_customers["Acquisition Channel"] = df_customers["Acquisition Channel"].astype('category')
df_customers["Acquisition Device"] = df_customers["Acquisition Device"].astype('category')

In [4]:
df_funnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
customerID            10000 non-null int64
year                  10000 non-null int64
month                 10000 non-null int64
class name            10000 non-null object
skus viewed           10000 non-null int64
skus added to cart    10000 non-null int64
skus purchased        10000 non-null int64
revenue generated     10000 non-null int64
year-month            10000 non-null object
dtypes: int64(7), object(2)
memory usage: 703.2+ KB


In [5]:
df_funnel['class name'] = df_funnel['class name'].astype('category')
df_funnel['year-month'] = df_funnel['year-month'].astype('datetime64')

The 'Customers' dataset describes when and how Wayfair acquired (email signed up) each customer. There are 1,383 total entries in the table after removing 19 rows with duplicate customerID values (note: for each duplicate pair, the row with more data was retained). While each customerID value has an Acquisition Date value, not all of them have values for Acquisition Channel or Acquisition Device. I initally thought to delete rows without an entry for both Channel and Device to save memory, but I decided not to as Acquisition Date alone can still yield good insights.

The 'Funnel' table describes the activity each customer has had with SKUs on the website and how much revenue was generated if they purchased. There are 10,000 entries, and each entry has a value in every column. This is great data! 

Now to decide the best way to join the tables into one dataframe. An inner join would join the data on the CustomerID values that existed in *both* tables. However, there are 8 more unique CustomerIDs in the Funnel table than in the Customer table, and we don't want to waste this valuable data. There also aren't any unique customerIDs in Customer that aren't already in Funnel. The left outer merge ('JOIN' in SQL) on the customerID column is best for this situation.

In [6]:
df_merged_inner = pd.merge(df_funnel, df_customers, how = 'inner', on = 'customerID')
df_merged_inner.info()
del df_merged_inner # it doesnt give us the full data, so this dataset isn't needed

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9992 entries, 0 to 9991
Data columns (total 12 columns):
customerID             9992 non-null int64
year                   9992 non-null int64
month                  9992 non-null int64
class name             9992 non-null category
skus viewed            9992 non-null int64
skus added to cart     9992 non-null int64
skus purchased         9992 non-null int64
revenue generated      9992 non-null int64
year-month             9992 non-null datetime64[ns]
Acquisition Date       9992 non-null datetime64[ns]
Acquisition Channel    7093 non-null category
Acquisition Device     3977 non-null category
dtypes: category(3), datetime64[ns](2), int64(7)
memory usage: 810.6 KB


In [7]:
df_merged = pd.merge(df_funnel, df_customers, how = 'left', on = 'customerID')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 12 columns):
customerID             10000 non-null int64
year                   10000 non-null int64
month                  10000 non-null int64
class name             10000 non-null category
skus viewed            10000 non-null int64
skus added to cart     10000 non-null int64
skus purchased         10000 non-null int64
revenue generated      10000 non-null int64
year-month             10000 non-null datetime64[ns]
Acquisition Date       9992 non-null datetime64[ns]
Acquisition Channel    7093 non-null category
Acquisition Device     3977 non-null category
dtypes: category(3), datetime64[ns](2), int64(7)
memory usage: 811.2 KB


In [8]:
df_merged.head(50)

,customerID,year,month,class name,skus viewed,skus added to cart,skus purchased,revenue generated,year-month,Acquisition Date,Acquisition Channel,Acquisition Device
0,21,2016,9,Accent Pillows,3,0,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
1,21,2016,9,Area Rugs,6,1,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
2,21,2016,9,Bar Stools,30,0,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
3,21,2016,9,Beds,27,4,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
4,21,2016,9,Curtains & Drapes,1,0,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
5,21,2016,9,End Tables,1,0,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
6,21,2016,9,Sheets And Sheet Sets,12,1,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
7,21,2016,9,TV Stands & Entertainment Centers,3,0,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
8,21,2016,9,Wall Art,1,1,0,0,2016-09-01,2008-06-14,Search - Paid,Desktop
9,3302,2016,9,Accent Pillows,1,0,0,0,2016-09-01,2012-05-29,NaN,Desktop


In [9]:
df_merged.to_excel('merged.xlsx') # for easier pivot table analysis